In [1]:
import os
import urllib
import traceback
import time
import sys
import numpy as np
import cv2
from utils import preproc as preprocess
from rknn.api import RKNN
import time
from utils import mkdir, multiclass_nms, demo_postprocess, vis
from config import Config


In [2]:
cfg = Config()
ONNX_MODEL = '/home/white/PycharmProjects/YOLOX-main/tools/yolox.onnx'
RKNN_MODEL = 'yolox.rknn'
input_shape = tuple(map(int, "640,640".split(',')))

In [3]:
rknn = RKNN()
rknn.config(mean_values=[0.485,0.456,0.406], std_values=[0.229,0.224,0.225],batch_size=50,epochs=4)


W config: Generally, when the model input is an image, the range of mean_values/std_values should be in [0, 255], but the value you filled is too small,
          please check carefully!
W config: epochs = '4' is not support currently!


0

In [4]:
ret = rknn.load_onnx(model=ONNX_MODEL)


In [5]:
ret = rknn.build(do_quantization=False,dataset='dataset.txt')
# ret = rknn.build(do_quantization=False)

Analysing : 100%|███████████████████████████████████████████████| 279/279 [00:00<00:00, 5089.40it/s]


In [6]:
ret = rknn.export_rknn(RKNN_MODEL)

In [7]:
origin_img = cv2.imread(cfg.image_path)

In [8]:
# mean = (0.485, 0.456, 0.406)
# std = (0.229, 0.224, 0.225)

In [9]:
img, ratio = preprocess(origin_img, input_shape,mean=None,std=None)

In [10]:
ret = rknn.init_runtime()

W init_runtime: target is None, use simulator!


In [11]:
outputs = rknn.inference(inputs=[img],data_format='nchw')

In [12]:
outputs[0].squeeze(axis=3).shape

(1, 8400, 12)

In [13]:
predictions = demo_postprocess(outputs[0].squeeze(axis=3), input_shape)[0]

[array([[[ 0,  0],
        [ 1,  0],
        [ 2,  0],
        ...,
        [77, 79],
        [78, 79],
        [79, 79]]]), array([[[ 0,  0],
        [ 1,  0],
        [ 2,  0],
        ...,
        [37, 39],
        [38, 39],
        [39, 39]]]), array([[[ 0,  0],
        [ 1,  0],
        [ 2,  0],
        [ 3,  0],
        [ 4,  0],
        [ 5,  0],
        [ 6,  0],
        [ 7,  0],
        [ 8,  0],
        [ 9,  0],
        [10,  0],
        [11,  0],
        [12,  0],
        [13,  0],
        [14,  0],
        [15,  0],
        [16,  0],
        [17,  0],
        [18,  0],
        [19,  0],
        [ 0,  1],
        [ 1,  1],
        [ 2,  1],
        [ 3,  1],
        [ 4,  1],
        [ 5,  1],
        [ 6,  1],
        [ 7,  1],
        [ 8,  1],
        [ 9,  1],
        [10,  1],
        [11,  1],
        [12,  1],
        [13,  1],
        [14,  1],
        [15,  1],
        [16,  1],
        [17,  1],
        [18,  1],
        [19,  1],
        [ 0,  2],
        [ 1, 

In [14]:
predictions[:2]

array([[7.71484375e+00, 1.01796875e+01, 2.28344345e+01, 3.43791199e+01,
        3.02124023e-03, 3.02124023e-03, 3.87573242e-03, 4.04052734e-02,
        1.48925781e-02, 3.78417969e-02, 1.59912109e-01, 1.32751465e-02],
       [1.58867188e+01, 7.03515625e+00, 3.45473938e+01, 2.40709095e+01,
        3.02124023e-03, 3.02124023e-03, 3.02124023e-03, 1.95617676e-02,
        9.49096680e-03, 2.84423828e-02, 1.22192383e-01, 1.94702148e-02]],
      dtype=float32)

In [15]:
boxes = predictions[:, :4]
scores = predictions[:, 4:5] * predictions[:, 5:]

In [16]:
boxes_xyxy = np.ones_like(boxes)

In [17]:
boxes[:4] # boxes is 

array([[ 7.7148438, 10.1796875, 22.834435 , 34.37912  ],
       [15.886719 ,  7.0351562, 34.547394 , 24.07091  ],
       [22.179688 ,  5.46875  , 34.818356 , 17.236336 ],
       [30.007812 ,  5.40625  , 36.811443 , 15.188747 ]], dtype=float32)

In [18]:
boxes_xyxy[:, 0] = boxes[:, 0] - boxes[:, 2]/2.
boxes_xyxy[:, 1] = boxes[:, 1] - boxes[:, 3]/2.
boxes_xyxy[:, 2] = boxes[:, 0] + boxes[:, 2]/2.
boxes_xyxy[:, 3] = boxes[:, 1] + boxes[:, 3]/2.
boxes_xyxy /= ratio

In [19]:
boxes_xyxy[:6]


array([[-2.8924794, -5.476463 , 14.946923 , 21.382225 ],
       [-1.0835767, -3.9064832, 25.906576 , 14.898914 ],
       [ 3.726961 , -2.4604828, 30.928802 , 11.005404 ],
       [ 9.064134 , -1.7094717, 37.823074 , 10.156737 ],
       [16.421934 , -1.5696913, 48.348576 , 11.609975 ],
       [17.522482 , -1.8616714, 53.559067 , 12.762549 ]], dtype=float32)

In [20]:
dets = multiclass_nms(boxes_xyxy, scores, nms_thr=0.6, score_thr=0.1)

In [21]:
dets

array([[105.96783447,  87.280159  , 425.77047729, 303.24719238,
          0.91576624,   4.        ]])

In [22]:
if dets is not None:
    final_boxes, final_scores, final_cls_inds = dets[:, :4], dets[:, 4], dets[:, 5]
    origin_img = vis(origin_img, final_boxes, final_scores, final_cls_inds,
                     conf=cfg.score_thr, class_names=cfg.COCO_CLASSES)

In [23]:
cv2.imshow("test",origin_img)

In [ ]:
cv2.waitKey(0)